In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
debit = pd.read_excel(r'debit_by_sector.xlsx')

debit = debit[debit['Дата'].dt.year > 2022]

debit_ind = debit[debit.columns[2:-3]].diff(periods=1).iloc[1:].reset_index(drop=True)
debit_dep_long = debit[debit.columns[-1]].diff(periods=1).iloc[1:].reset_index(drop=True)
debit_dep_short = debit[debit.columns[-3]].diff(periods=1).iloc[1:].reset_index(drop=True) + debit[debit.columns[-2]].diff(periods=1).iloc[1:].reset_index(drop=True)

In [ ]:
# Define the objective function (MSE) using DataFrame columns
def mse(params):
    y_pred_long = (debit_ind * params).T.sum()
    y_pred_short = (debit_ind * [1 - x for x in params]).T.sum()
    return np.mean((debit_dep_long - y_pred_long) ** 2) + np.mean((debit_dep_short - y_pred_short) ** 2)

# Initial guess for parameters
initial_guess = [0.5] *int(len(debit_ind.columns))  # Starting parameters within [0, 1]

# Set the parameter bounds to be between 0 and 1
bounds = [(0, 0.9)]*int(len(debit_ind.columns))


# Minimize the MSE with parameter constraints using scipy
result = minimize(mse, 
                  initial_guess, 
                  bounds=bounds,
                  method='trust-constr'
                 )

# Get the optimized parameters
optimal_params = result.x
pd.concat([pd.DataFrame(debit_ind.columns.values).T, pd.DataFrame(optimal_params).round(decimals=3).T]).T

In [ ]:
x = debit['Дата'][1:]
y = debit_dep_long
 
# first plot with X and Y data
plt.plot(x, y, label=debit.columns[-1])
 
x1 = debit['Дата'][1:]
y1 = (debit_ind*optimal_params).T.sum()
 
# second plot with x1 and y1 data
plt.plot(x1, y1, '-.', label='prediction')

plt.legend()
plt.xlabel("X-axis data")
plt.ylabel("Y-axis data")
plt.title('multiple plots')
plt.show()